# model.ops
본 노트에서는 [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882) 논문을 예시로 활용하여, 딥러닝 모형 구현을 위한 각종 연산들을 모듈화하는 방법에 대해서 다룹니다. 전체 딥러닝 모형을 구현하기 위해 필요한 연산들은 따로 하나의 script (eg. `ops.py`)에 모아놓는 것이 좋습니다.  이렇게 하나의 script에 모아놓은 snippet은 전체 딥러닝 모형을 구현하는 script (eg. `net.py`)에서 활용합니다. 이때 필요한 연산을 선정하여 구현하는 기준은 대략적으로 아래처럼 생각해 볼 수 있습니다.

1. DeepLearning framework에서 제공하지 않을 경우 (이미 있는 것을 구현하지 말 것)
2. DeepLearning framework에서 이미 제공하지만 무언가 기능이 필요할 경우 
3. 전체 딥러닝 모형을 정리하는 script에서 코드가 간결해 질 수 있도록 (보기쉬운 코드가 좋은 코드)


상기된 논문의 구조는 대략적으로 아래와 같습니다. 이를 보아 대략적으로 아래와 같은 연산들만 구현하여 모듈화를 하면 코드가 간결해 질 것 입니다.

* `ops.py`: `MultiChannelEmbedding` class, `ConvolutionLayer` class, `MaxOverTimepooling` class
* `net.py`: `SenCNN` class
![Alt text](https://raw.githubusercontent.com/aisolab/strnlp/master/materials/img/sencnn_structure.png)

### Implementation tips
**위와 같은 연산을 가장 쉽게 구현하는 방법 중 하나는 예제데이터를 미리 생각해놓고 확인하면서 구현하는 것입니다.** 딥러닝 모형 학습 시 대게 미니배치를 이용하여 학습하므로, 우리는 구현해야할 연산이 가장 간단한 미니배치(eg. 미니배치 사이즈가 2인 경우)를 상정하고 각각의 연산을 구현합니다. 이를 위해서 `model.data.ipynb` 노트에서 활용했던 코드들을 가져와서 가장 간단한 미니배치를 구성합니다.

#### `Corpus` class의 instance 생성

In [1]:
import pickle
import torch
from pathlib import Path
from pprint import pprint
from mecab import MeCab
from torch.utils.data import DataLoader
from model.data import Corpus
from model.utils import Vocab, Tokenizer, PadSequence

data_dir = Path.cwd() / 'data'
list_of_dataset = list(data_dir.iterdir())
pprint(list_of_dataset)
with open(list_of_dataset[-1], mode='rb') as io:
    vocab = pickle.load(io)

pad_sequence = PadSequence(length=32, pad_val=vocab.to_indices(vocab.padding_token))
split_morphs = MeCab().morphs
tokenizer = Tokenizer(vocab, split_fn=split_morphs, pad_fn=pad_sequence)

[PosixPath('/root/Documents/archive/strnlp/exercise/data/.DS_Store'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/train.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/morphs_vec.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/validation.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/tokenizer.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/test.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/vocab.pkl')]


In [2]:
tr_corpus = Corpus(list_of_dataset[1], transform_fn=tokenizer.split_and_transform)

#### `DataLoader` class의 instance를 이용, 가장 간단한 미니배치 구성

In [3]:
tr_dl = DataLoader(tr_corpus, batch_size=2, shuffle=True)

In [4]:
mb = next(iter(tr_dl))
x_mb, y_mb = mb

print(mb)
print(x_mb, y_mb)

[tensor([[3070, 2750, 3116, 3109, 2750, 2084,  782, 3111, 4657,  776, 5458, 2924,
            0, 6172, 8439, 5358, 2370, 1901, 2227, 2370, 2251, 5182, 1901, 7485,
         6722, 6204, 3900, 8391,    0, 1953, 6219, 2251],
        [1160, 5294, 6219,  471, 5442, 1901, 5657, 5316, 6214, 6422, 3334, 5294,
         3753,  864, 4917, 1953, 1680, 6408, 7564, 5658, 5479, 6591, 6512, 1901,
         7461,    1,    1,    1,    1,    1,    1,    1]]), tensor([0, 0])]
tensor([[3070, 2750, 3116, 3109, 2750, 2084,  782, 3111, 4657,  776, 5458, 2924,
            0, 6172, 8439, 5358, 2370, 1901, 2227, 2370, 2251, 5182, 1901, 7485,
         6722, 6204, 3900, 8391,    0, 1953, 6219, 2251],
        [1160, 5294, 6219,  471, 5442, 1901, 5657, 5316, 6214, 6422, 3334, 5294,
         3753,  864, 4917, 1953, 1680, 6408, 7564, 5658, 5479, 6591, 6512, 1901,
         7461,    1,    1,    1,    1,    1,    1,    1]]) tensor([0, 0])


### `MultiChannelEmbedding`, `ConvolutionLayer`, `MaxOverTimepooling` 구현

#### Load modules

In [2]:
import torch
import torch.nn as nn
from model.ops import MultiChannelEmbedding, ConvolutionLayer, MaxOverTimePooling

#### `MultiChannelEmbedding` class

In [6]:
# nn.Embedding.from_pretrained, torch.Tensor.permute를 이용하여 직접 구현해보세요.
# https://github.com/aisolab/strnlp/blob/master/exercise/model/ops.py
class MultiChannelEmbedding(nn.Module):
    """MultiChannelEmbedding class"""
    def __init__(self, vocab: Vocab) -> None:
        """Instantiating MultiChannelEmbedding class

        Args:
            vocab (model.utils.Vocab): the instance of model.utils.Vocab
        """
        super(MultiChannelEmbedding, self).__init__()
        pass

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        pass

In [7]:
ops1 = MultiChannelEmbedding(vocab)
h1 = ops1(x_mb)
pprint(h1)

(tensor([[[-0.2286, -0.2849, -0.5418,  ..., -0.3589, -0.0767,  0.0862],
         [ 0.1945, -0.0632,  0.0037,  ...,  0.2641,  0.0233,  0.2906],
         [-0.7672, -0.3738, -0.3894,  ..., -0.6365, -0.2963, -0.4155],
         ...,
         [-0.2776,  0.3091, -0.8164,  ...,  0.2565,  0.2070,  0.0206],
         [-0.2625, -0.0234,  0.2058,  ...,  0.1482, -0.0095, -0.3524],
         [-0.0459,  0.1627, -0.5181,  ..., -0.0044,  0.1284,  0.5323]],

        [[-0.5872, -0.6691, -0.0767,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.2138,  0.6100,  0.0233,  ...,  0.0000,  0.0000,  0.0000],
         [-0.7448, -0.9603, -0.2963,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.2106,  0.3919,  0.2070,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.1595, -0.2109, -0.0095,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.2779,  0.3197,  0.1284,  ...,  0.0000,  0.0000,  0.0000]]]),
 tensor([[[-0.2286, -0.2849, -0.5418,  ..., -0.3589, -0.0767,  0.0862],
         [ 0.1945, -0.0632,  0.0037,  ...,

In [8]:
print(h1[0].shape, h1[1].shape)

torch.Size([2, 300, 32]) torch.Size([2, 300, 32])


### `ConvolutionLayer` class 

In [9]:
# nn.Conv1d, F.relu를 이용하여 직접 구현해보세요.
# https://github.com/aisolab/strnlp/blob/master/exercise/model/ops.py
class ConvolutionLayer(nn.Module):
    """ConvolutionLayer class"""
    def __init__(self, in_channels: int, out_channels: int) -> None:
        """Instantiating ConvolutionLayer class

        Args:
            in_channels (int): the number of channels from input feature map
            out_channels (int): the number of achannels from output feature map
        """
        super(ConvolutionLayer, self).__init__()
        pass
    
    def forward(self, x: Tuple[torch.Tensor, torch.Tensor]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        pass

In [10]:
ops2 = ConvolutionLayer(300, 300)
h2 = ops2(h1)
pprint(h2)

(tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0417, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.1638, 0.2253,  ..., 0.1884, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0152, 0.0844,  ..., 0.0000, 0.0404, 0.1249],
         [0.0000, 0.1207, 0.0825,  ..., 0.0746, 0.0000, 0.0000],
         [0.1338, 0.2159, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0108, 0.0108, 0.0108],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.1776,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.3244,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.3411,  ..., 0.0163, 0.0163, 0.0163],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       grad_fn=<AddBackward0>),
 tensor([[[0.0000, 0.0000, 0.6283,  ..., 0.1075, 0.0751, 0.0420],
         [0.0000, 0.0000, 0.1336,  ..., 0.0000, 0.0000, 0.0000],
         [0.0873, 0.0000

In [11]:
print(h2[0].shape, h2[1].shape, h2[2].shape)

torch.Size([2, 100, 30]) torch.Size([2, 100, 29]) torch.Size([2, 100, 28])


#### `MaxOverTimePooling` class

In [12]:
# torch.cat을 이용하여 직접 구현해보세요.
# https://github.com/aisolab/strnlp/blob/master/exercise/model/ops.py
class MaxOverTimePooling(nn.Module):
    """MaxOverTimePooling class"""
    def forward(self, x: Tuple[torch.Tensor, torch.Tensor, torch.Tensor]) -> torch.Tensor:
        pass

In [13]:
ops3 = MaxOverTimePooling()
h3 = ops3(h2)
pprint(h3)

tensor([[0.2640, 0.2315, 0.8197, 0.2081, 0.4671, 0.2751, 0.5050, 0.5101, 0.4386,
         0.5332, 0.5659, 0.1766, 0.6489, 0.4278, 0.3121, 0.8617, 0.8794, 0.6339,
         0.7556, 0.5185, 0.1264, 0.4284, 0.8094, 0.7758, 0.4959, 0.6870, 0.7204,
         0.4188, 0.9218, 0.5356, 0.4754, 0.5462, 0.1303, 0.0880, 0.4976, 0.1217,
         0.4644, 0.2966, 0.3623, 0.6821, 0.5480, 0.8581, 0.5285, 0.6904, 0.4506,
         0.7266, 0.3949, 0.6736, 0.6757, 0.4294, 0.3831, 0.2945, 1.0985, 0.4672,
         0.7046, 0.0690, 0.5930, 0.5998, 0.5665, 0.2038, 0.8935, 0.5419, 0.4419,
         0.0232, 0.4533, 0.4174, 0.3695, 0.5994, 0.3664, 0.2113, 0.4901, 0.5019,
         0.5643, 0.0317, 0.4696, 0.0000, 0.2428, 0.4474, 0.0618, 0.3768, 0.6045,
         0.7579, 0.5252, 0.4199, 0.4577, 0.3627, 0.4137, 0.2940, 0.2348, 0.5498,
         0.5365, 0.4529, 0.5545, 0.4880, 0.5185, 0.2144, 0.4718, 0.2025, 0.1990,
         0.2343, 0.6283, 0.1336, 0.7006, 0.3720, 0.2310, 0.7196, 0.1828, 0.8884,
         0.9363, 0.5323, 0.7

In [14]:
print(h3.shape)

torch.Size([2, 300])
